# Programming Assignment: Настройка параметров
В этом задании вам предстоит поэкспериментировать с параметрами вашей модели для сентимент-анализа. Все задания выполняются на том же датасете, что и на прошлой неделе.

## Инструкции

In [1]:
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [2]:
def write_result(file_name, result):
    file_obj = open(file_name, "w")
    file_obj.write(result)
    file_obj.close()

In [3]:
negids = movie_reviews.fileids("neg")
posids = movie_reviews.fileids("pos")
negfeats = [movie_reviews.raw(fileids=[f]) for f in negids]
posfeats = [movie_reviews.raw(fileids=[f]) for f in posids]
neg_classes = [0] * len(negids)
pos_classes = [1] * len(posids)
classes = neg_classes + pos_classes
all_texts = negfeats + posfeats

In [4]:
results_columns = ["Accuracy"]
results_frame = pd.DataFrame(columns=results_columns)

1.Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

In [5]:
cv = 5

In [6]:
logistic_count_pipe = Pipeline([("vectorize", CountVectorizer()), ("model", LogisticRegression())])
logistic_tfidf_pipe = Pipeline([("vectorize", TfidfVectorizer()), ("model", LogisticRegression())])

In [7]:
count_score = cross_val_score(logistic_count_pipe, all_texts, classes, cv=cv)
tfidf_score = cross_val_score(logistic_tfidf_pipe, all_texts, classes, cv=cv)

In [8]:
scores_str = "{0} {1} {2} {3}".format(
    count_score.mean(),
    count_score.std(),
    tfidf_score.mean(),
    tfidf_score.std())
write_result("scores.txt", scores_str)
results_frame = results_frame.append(pd.DataFrame([
    [count_score.mean()],
    [tfidf_score.mean()]
], index=["Count Vectorizer Logistic", "Tfidf Vectorizer Logistic"], columns=results_columns))
print(scores_str)

0.841 0.01677796173556255 0.8210000000000001 0.004062019202317978


2.Попробуйте позадавать параметр min_df у CountVectorizer. Оцените качество вашего классикатора с min_df=10 и с min_df=50.

In [9]:
min_df_10_count_pipe = Pipeline([("vectorize", CountVectorizer(min_df=10)), ("model", LogisticRegression())])
min_df_50_count_pipe = Pipeline([("vectorize", CountVectorizer(min_df=50)), ("model", LogisticRegression())])

In [10]:
min_df_10_score = cross_val_score(min_df_10_count_pipe, all_texts, classes, cv=cv)
min_df_50_score = cross_val_score(min_df_50_count_pipe, all_texts, classes, cv=cv)

In [11]:
min_df_scores = [min_df_10_score.mean(), min_df_50_score.mean()]
min_df_scores_str = "{0} {1}".format(min_df_scores[0], min_df_scores[1])
write_result("min_df_scores.txt", min_df_scores_str)
results_frame = results_frame.append(pd.DataFrame(
    list(map(lambda score: [score], min_df_scores)),
    index=["min_df_10 Count Vectorizer Logistic", "min_df_50 Count Vectorizer Logistic"], columns=results_columns))
print(min_df_scores_str)

0.8390000000000001 0.813


3.Поперебирайте классификатор, используемый после CountVectorizer. И vectorizer и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [12]:
svm_count_pipe = Pipeline([("vectorize", CountVectorizer()), ("model", LinearSVC())])
sgd_count_pipe = Pipeline([("vectorize", CountVectorizer()), ("model", SGDClassifier())])

In [13]:
svm_count_score = cross_val_score(svm_count_pipe, all_texts, classes, cv=cv)
sgd_count_score = cross_val_score(sgd_count_pipe, all_texts, classes, cv=cv)
model_scores = [count_score.mean(), svm_count_score.mean(), sgd_count_score.mean()]
write_result("min_model_score.txt", str(min(model_scores)))
results_frame = results_frame.append(pd.DataFrame([
    [svm_count_score.mean()],
    [sgd_count_score.mean()]
], index=["Count Vectorizer SVM", "Count Vectorizer SGD"], columns=results_columns))
print("\n".join(
    map(
        lambda mdl: "{0}: {1}".format(mdl[0], mdl[1]),
        zip(
            ["Logistic", "SVM", "SGD"],
            model_scores
))))

C:\Users\agasp\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\agasp\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\agasp\Anaconda3\lib\sit

Logistic: 0.841
SVM: 0.8325000000000001
SGD: 0.7565


4.Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [14]:
# nltk.download("stopwords")
english_stopwords = stopwords.words("english")
nltk_stopwords_pipe = Pipeline([
    ("vectorize", CountVectorizer(stop_words=english_stopwords)),
    ("model", LogisticRegression())
])
sklearn_stopwords_pipe = Pipeline([
    ("vectorize", CountVectorizer(stop_words="english")),
    ("model", LogisticRegression())
])

In [15]:
nltk_stopwords_score = cross_val_score(nltk_stopwords_pipe, all_texts, classes, cv=cv)
sklearn_stopwords_score = cross_val_score(sklearn_stopwords_pipe, all_texts, classes, cv=cv)
stopwords_scores = [nltk_stopwords_score.mean(), sklearn_stopwords_score.mean()]
write_result("stopwords_scores.txt", " ".join(map(str, stopwords_scores)))
results_frame = results_frame.append(pd.DataFrame(
    list(map(lambda score: [score], stopwords_scores)),
    index=["Nltk stopwords Count Vectorizer Logistic", "Sklearn stopwords Count Vectorizer Logistic"],
    columns=results_columns))
print("\n".join(map(
    lambda model: "{0}: {1}".format(model[0], model[1]),
    zip(
        ["nltk", "sklearn"],
        stopwords_scores)
)))

nltk: 0.8414999999999999
sklearn: 0.8390000000000001


5.Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [16]:
bigram_pipe = Pipeline([
    ("vectorize", CountVectorizer(analyzer="word")),
    ("model", LogisticRegression())
])
bigram_score = cross_val_score(bigram_pipe, all_texts, classes, cv=cv)

In [17]:
ngram_range_pipe = Pipeline([
    ("vectorize", CountVectorizer(analyzer="char_wb", ngram_range=(3, 5))),
    ("model", LogisticRegression())
])
ngram_range_score = cross_val_score(ngram_range_pipe, all_texts, classes, cv=cv)

In [18]:
ngram_scores = list(map(lambda scores: scores.mean(), [bigram_score, ngram_range_score]))
write_result("ngram_scores.txt", " ".join(map(str, ngram_scores)))
results_frame = results_frame.append(pd.DataFrame(
    list(map(lambda score: [score], ngram_scores)),
    index=["Bigrams Count Vectorizer Logistic", "Ngrams (3,5) Count Vectorizer Logistic"],
    columns=results_columns))
print("\n".join(
    map(
        lambda model: "{0}: {1}".format(model[0], model[1]),
        zip(
            ["Bigram", "Ngram (3,5)"],
            ngram_scores))))

Bigram: 0.841
Ngram (3,5): 0.819


In [21]:
results_frame.sort_values(results_columns, ascending=False)

,Accuracy
Nltk stopwords Count Vectorizer Logistic,0.8415
Count Vectorizer Logistic,0.8410
Bigrams Count Vectorizer Logistic,0.8410
min_df_10 Count Vectorizer Logistic,0.8390
Sklearn stopwords Count Vectorizer Logistic,0.8390
Count Vectorizer SVM,0.8325
Tfidf Vectorizer Logistic,0.8210
"Ngrams (3,5) Count Vectorizer Logistic",0.8190
min_df_50 Count Vectorizer Logistic,0.8130
Count Vectorizer SGD,0.7565
